In [14]:
!pip install seaborn | wc -l
!pip install matplotlib | wc -l

14
11


In [1]:
CLASS_LIST_29 = ["Tee","Tank","Dress","Shorts","Skirt","Jumpsuit","Sweater","Blazer","Striped","Cardigan","Blouse","Romper","Sweatpants","Jacket", "Jeans", "Poncho", "Button-Down", "Pencil", "Maxi",  "Floral", "Pleated", "Mesh","Bandana", "Tie-Dye", "Culottes", "Embroidered", "Kimono","Chevron", "Buttoned"]
CLASS_LIST_35 = ["Tee","Tank","Dress","Shirt","Shorts","Skirt","Hoodie","Jumpsuit","Cargo","Turtleneck","Sweater","Plaid","Blazer","Striped","Cardigan","Blouse","Romper","Jacket","Jeans","Maxi","Floral","Denim","Trench","Baroque","Ornate","Belted","Jersey","Sweatshorts","PJ","Parka","Polka","Beaded","Shawl","Bodycon","Abstract"]
CLASS_LIST_21 = ["Tee","Tank","Dress","Shorts","Skirt","Hoodie","Jumpsuit","Sweater","Blazer","Striped","Cardigan","Blouse","Jacket","Jeans","Maxi","Floral","Denim","Sweatshorts","Polka","Shawl","Bodycon"]
CLASS_LIST = CLASS_LIST_21

for idx, val in enumerate(CLASS_LIST):
    print("%i/%s" % (idx, val))

0/Tee
1/Tank
2/Dress
3/Shorts
4/Skirt
5/Hoodie
6/Jumpsuit
7/Sweater
8/Blazer
9/Striped
10/Cardigan
11/Blouse
12/Jacket
13/Jeans
14/Maxi
15/Floral
16/Denim
17/Sweatshorts
18/Polka
19/Shawl
20/Bodycon


In [2]:
import tensorflow as tf
import keras.backend as tfb
import numpy as np

POS_WEIGHT = 10  # multiplier for positive targets, needs to be tuned

def weighted_binary_crossentropy(target, output):
    """
    Weighted binary crossentropy between an output tensor 
    and a target tensor. POS_WEIGHT is used as a multiplier 
    for the positive targets.

    Combination of the following functions:
    * keras.losses.binary_crossentropy
    * keras.backend.tensorflow_backend.binary_crossentropy
    * tf.nn.weighted_cross_entropy_with_logits
    """
    # transform back to logits
    _epsilon = tfb._to_tensor(tfb.epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
    output = tfb.log(output / (1 - output))
    # compute weighted loss
    loss = tf.nn.weighted_cross_entropy_with_logits(labels=target,
                                                    logits=output,
                                                    pos_weight=POS_WEIGHT)
    return tf.reduce_mean(loss, axis=-1)

Click below to import f1 function

In [3]:
import tensorflow.keras.backend as K
import tensorflow as tf
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [5]:
import tensorflow.keras as keras
import os
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
os.chdir('C:\\Users\\mdc20')
from keras.models import load_model
model = load_model('best_model_21_class_lr@0.002-1666761586.9275644.hdf5',custom_objects={'weighted_binary_crossentropy':weighted_binary_crossentropy, 'f1_loss':f1_loss, 'f1':f1})

In [6]:
datasetdir = 'E:\\img_highres'
os.chdir(datasetdir)

from msilib.schema import Directory
from random import shuffle
import tensorflow as tf
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input


from tensorflow.keras.preprocessing.image import ImageDataGenerator
BATCH_SIZE = 16
IMG_SHAPE = (224,224)


from collections import defaultdict
import pandas as pd
datasetdir = 'E:\\img_highres'
def constructImageClassDataFrame(shape, min_class_occurence = 1, class_list = None):
    # Args: 
    #   Shape: Image shape (2D)
    #   min_class_occurence: number of times class must occur in labeled dir in order to add it to the final sclass list
    sub_dirs = [d for d in os.listdir(os.getcwd()) if os.path.isdir(os.path.join(datasetdir, d))]
    
    if class_list is None:
        classes = defaultdict(lambda: 0)

        for sub_dir in sub_dirs:
            labels = sub_dir.split('_')
            for label in labels:
                classes[label] += 1
        top_k_classes = [cls for cls in classes if classes[cls] >= min_class_occurence]
        label_classes = top_k_classes    
    else:
        classes = class_list
        label_classes = classes

    
    arr = [[labeled_dir, file] for labeled_dir in sub_dirs for file in os.listdir(labeled_dir) ]
    print("Done parsing through directories")
    df = pd.DataFrame(data=arr, columns=["folder","filename"])



    df['filename'] = df['folder'] + '/' + df['filename']
    df['labels'] = df['folder'].apply(lambda x : [y for y in x.split('_') if y in label_classes] if len([y for y in x.split('_') if y in label_classes]) > 0 else None)

    print(np.shape(df))
    df = df[df.labels.notnull()]

    return df, label_classes

    

def DataLoad(shape, preprocessing): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        horizontal_flip = True, 
        validation_split = 0.5,
        rescale = 1.0/255, 
    )

    height, width = shape

    df, classes = constructImageClassDataFrame(shape, class_list=CLASS_LIST)

    # Modify dataframe to only take images with references to classes

    classes = list(classes)
    print("Num classes #")
    print(len(classes)) 
    # for subdir

    # train_dataset = imgdatagen.flow_from_dataframe(
    #     dataframe = df,
    #     directory = datasetdir,
    #     x_col="filename",
    #     y_col="labels",
    #     batch_size = batch_size,
    #     seed = 116,
    #     shuffle = True,
    #     class_mode="categorical",
    #     classes = classes,
    #     subset = 'training'
    # )
    val_dataset = imgdatagen.flow_from_dataframe(
        dataframe = df,
        directory = datasetdir,
        x_col="filename",
        y_col="labels",
        batch_size = BATCH_SIZE,
        seed = 42,
        target_size = IMG_SHAPE,
        shuffle = True,
        class_mode="categorical",
        classes = classes,
        subset = 'validation'
    )


    return 0, val_dataset, df

_, val_dataset, df = DataLoad((224,224), preprocessing=preprocess_input)


Done parsing through directories
(289212, 3)
Num classes #
21
Found 126798 validated image filenames belonging to 21 classes.


In [6]:
import sklearn.metrics as skm
def get_indices_from_keras_generator(gen, batch_size):
    """
    Given a keras data generator, it returns the indices and the filepaths
    corresponding the current batch. 
    :param gen: keras generator.
    :param batch_size: size of the last batch generated.
    :return: tuple with indices and filenames
    """

    idx_left = (gen.batch_index - 1) * batch_size
    idx_right = idx_left + gen.batch_size if idx_left >= 0 else None
    indices = gen.index_array[idx_left:idx_right]
    filenames = [gen.filenames[i] for i in indices]
    return indices, filenames
from io import StringIO
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report

def get_confusion_matrix(y_test, y_hat, class_list, threshold = 0.5):
    # y_test is of form E = [e..e] where |E|=12 and e\in {0,1}
    # y_hat is of form P = [x..x] where 0 < x < 1 where P_x indicates probability of feature
    y_hat = np.where(y_hat > threshold,1,0)
    cm = confusion_matrix(y_test,y_hat,labels=class_list)
    return cm
# X_test, y_test = next(val_dataset)
# y_hat = model.predict(X_test)

class_list=CLASS_LIST

conf_matrix = np.array(np.zeros((14,2,2)))
print(np.shape(conf_matrix))
predicted_class_df = pd.DataFrame(columns=["filepath","class"])
MAX_ITERS = 500
it = 0
results_df = pd.DataFrame(columns=['class_identifier','precision','recall','f1-score','support','batch_number'],dtype=float)
for X_test,y_test in val_dataset:
    if it > MAX_ITERS:
        break
    if it % 10 == 0:
        print("%i/%i" % (it, MAX_ITERS))
    it = it + 1
    indices, filenames = get_indices_from_keras_generator(val_dataset,batch_size=4)
    y_hat = model.predict(X_test,verbose=0)
    y_hat = np.where(y_hat > 0.5,1,0)
    _
    # y_test_labels = np.argmax(y_test,axis=1)
    # y_hat_labels = np.argmax(y_hat, axis=1)

    # y_test_cm_input = [class_list[idx] for idx in y_test_labels]
    # y_hat_cm_input = [class_list[idx] for idx in y_hat_labels]
    cm = multilabel_confusion_matrix(y_test,y_hat,labels=range(0,14))
    # cm = get_confusion_matrix(y_test, y_hat, class_list=class_list)
    conf_matrix = np.add(conf_matrix, cm)
    # report = classification_report(y_test,y_hat,zero_division=0)
    # print(report)
    # rep_arr = report.split('\n')
    # rep_header = rep_arr[0].split()
    # rep_body = [row.split() for row in rep_arr[1:25]][1:]
# 
    # results_df.concat(rep_body)
    # df = pd.DataFrame(data=rep_body, columns=['class_identifier','precision','recall','f1-score','support'], index=class_list,dtype=float)
    # df['batch_number'] = it
    # results_df = pd.concat([results_df, df])

for class_name, conf_matrix in zip(class_list,conf_matrix):
    print(class_name)
    print(conf_matrix)
# results_df.sample(n=5)

(14, 2, 2)
0/500


KeyboardInterrupt: 

In [18]:

# results_df = results_df.drop(results_df[results_df.support == '0'].index)

# filtered_df = results_df[results_df.support != '0']
print(len(results_df))
# filtered_df.describe()

0


In [7]:
# Protect against DecompressionBomb Gating halting the training
import PIL
PIL.Image.MAX_IMAGE_PIXELS = None


y_pred = model.predict(val_dataset)


7925/7925 [==============================] - 1069s 134ms/step


In [8]:
print(np.shape(y_pred))


(126798, 21)


In [9]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=range(0,len(CLASS_LIST)), sparse_output=False)
y_true = val_dataset.classes
y_true_ohe = mlb.fit_transform(y_true)
print(np.shape(y_true_ohe))

(50719, 21)


In [9]:
def binarizeOutput(y_pred, threshold=0.5):
    if isinstance(threshold, list):
        if (len(threshold) == len(y_pred[0])):
            compare_to = [threshold for i in range(0, len(y_pred))]
            y_pred_bin  = y_pred > compare_to
    else:
        y_pred_bin = np.where(y_pred > threshold,1,0)
    return y_pred_bin



In [11]:
test_threshold = [0.35, 0.25, 0.45, 0.2, 0.15, 0.075, 0.15, 0.15, 0.2, 0.1, 0.2, 0.05, 0.05, 0.275, 0.15, 1.00, 1.00, 1.00, 0.2, 0.15, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00]
y_pred_bin = binarizeOutput(y_pred, threshold=0.5)


In [12]:
print(np.shape(y_pred_bin))
y_sum = np.sum(y_pred_bin,axis=1)
print(np.shape(y_sum))
y_sum_peek = y_sum[100:200]
print(y_sum_peek)
print(np.sum(y_sum))


(126798, 21)
(126798,)
[1 2 1 3 1 2 1 1 1 0 1 2 1 1 2 2 2 2 2 1 2 2 2 2 2 1 1 1 0 3 2 2 1 2 1 1 1
 1 1 1 1 1 2 1 2 0 2 2 1 1 1 1 1 1 1 2 1 1 1 1 0 0 1 1 1 1 2 2 3 2 1 2 1 2
 2 1 3 2 1 1 1 1 1 2 2 2 2 1 1 1 1 1 1 2 2 2 3 1 0 1]
174805


 ### Classes
0/Tee
1/Tank
2/Dress
3/Shorts
4/Skirt
5/Jumpsuit
6/Sweater
7/Blazer
8/Striped
9/Cardigan
10/Blouse
11/Romper
12/Sweatpants
13/Jacket
14/Jeans
15/Poncho
16/Button-Down
17/Pencil
18/Maxi
19/Floral
20/Pleated
21/Mesh
22/Bandana
23/Tie-Dye
24/Culottes
25/Embroidered
26/Kimono
27/Chevron
28/Buttoned

In [13]:
from sklearn.metrics import classification_report
SEARCH_THRESHOLD = True
t = 0.4
print("              precision    recall  f1-score   support")
for threshold in [0.05, 0.10, 0.15, 0.20, 0.225, 0.25, 0.275, 0.3, 0.35, 0.4,0.45, 0.5]:
    if(not SEARCH_THRESHOLD): break
    y_pred_bin = binarizeOutput(y_pred, threshold=threshold)    

    c_report = classification_report(y_true=y_true_ohe, y_pred=y_pred_bin,zero_division=False)
    # sample_avg =c_report.split('\n')[35]
    # print(c)
    # print(sample_avg)
    print("threshold:%f", threshold)
    print(c_report)


# y_pred_bin = binarizeOutput(y_pred, threshold=test_threshold)    
# c_report = classification_report(y_true=y_true_ohe, y_pred=y_pred_bin,zero_division=False)
# print(c_report)

              precision    recall  f1-score   support


In [11]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# print(y_test)
y_test_labels = np.argmax(y_test,axis=1)
y_hat_labels = np.argmax(y_hat, axis=1)

# print(np.shape(y_test_labels))
# print(np.shape(y_hat_labels))
# cm = confusion_matrix(y_test_labels,y_hat_labels,labels=class_list)
df_cm = pd.DataFrame(conf_matrix, index = class_list, columns = class_list)
plt.figure(figsize = (10,7))
sns.heatmap(df_cm,annot=True)

ValueError: Must pass 2-d input. shape=(14, 2, 2)

In [ ]:
np.shape(val_dataset.filenames)

In [ ]:

indices, filenames = get_indices_from_keras_generator(val_dataset, 1024)
print(np.shape(indices))
print(np.shape(filenames))
print(indices[0])
print(filenames[0:4])